

Importing necessary libraries

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
from statsmodels.tsa.stattools import coint
import seaborn as sns
import winsound
import warnings

Import price data, remove unnecessary coulmns

In [19]:
df = pd.read_csv("Prices.csv")
df.drop(["Name"],axis=1,inplace=True)

Convert the dates to standard datetime format

In [20]:
df['date']= pd.to_datetime(df['date'],dayfirst=True)
#df.info()
#len(df)
#df.head()

Set period for identifying pairs and testing the returns. Here we take one financial year.
Create a new dataframe (df1) contining only prices from the selected period.

In [21]:
df1=df.copy()
start=datetime.datetime(2008, 1, 1)
end=datetime.datetime(2019, 11, 30)

#for i in range(len(df1)):
#    if i%10000==0:
#        print(i)
#    if df1.iloc[i]['date']<start or df1.iloc[i]['date']>end:
#        indices.append(i)
#df1.drop(df1.index[indices],inplace=True)


indices=[]

df1 = df1[df1['date']>=start]
df1 = df1[df1['date']<end]


#print(len(df1))
#winsound.Beep(500,1000)
df1.reset_index(inplace=True)
df1.drop('index',axis=1,inplace=True)
df1.drop('PX_VOLUME',axis=1,inplace=True)

Removing those companies which have not traded for all the days in the 2008 to 2019 period.

In [24]:
df_s=df1.groupby('Ticker').count()
df_s.reset_index(inplace=True)

max=0
for i in range (len(df_s)):
    if df_s.iloc[i]['date']>max:
        max=df_s.iloc[i]['date']
#print(max)

less_dates=[]
for i in range (len(df_s)):
    if df_s.iloc[i]['date']!=max:
        print (df_s.iloc[i]['Ticker'])
        print (df_s.iloc[i]['date'])
        less_dates.append(df_s.iloc[i]['Ticker'])
print(less_dates)

for j in less_dates:
    df1 = df1[df1['Ticker']!=j]
#winsound.Beep(500,1000)

BAF IS Equity
2945
BHIN IS Equity
1710
BJAUT IS Equity
2849
BJFIN IS Equity
2848
BRIT IS Equity
2945
COAL IS Equity
2246
NEST IS Equity
2453
['BAF IS Equity', 'BHIN IS Equity', 'BJAUT IS Equity', 'BJFIN IS Equity', 'BRIT IS Equity', 'COAL IS Equity', 'NEST IS Equity']


Making a list of start dates of all the months in the whole 11 year period

In [29]:
start_date=pd.date_range('2008-01-01','2019-11-30', freq='MS').strftime("%Y-%b-%d").tolist()
for i in range (len(start_date)):
    start_date[i]= pd.to_datetime(start_date[i],dayfirst=True)

main pair forming code

In [28]:
final_pairs={}
for i in range (len(start_date)):
    
    df_dummy=df1.copy()
    
    #take start and end date for pair forming period
    
    if i <12:
        continue
    else:
        start = start_date[i-12]
        end = start_date[i]
    date=[]

    
    print("Start dates taken!")
    
    #remove all dates other than pair forming period
    
    df_dummy = df_dummy[df_dummy['date']>=start]
    df_dummy = df_dummy[df_dummy['date']<end]
    print('Other dates removed from df_dummy')
    
    #some necessary steps
    tickers=df_dummy['Ticker'].unique().tolist()
    
    date_list=[]
    
    df_forDate=df_dummy.groupby('date').count()
    df_forDate.reset_index(inplace=True)
    for k in range (len(df_forDate)):
        date_list.append(df_forDate.iloc[k]['date'])
    
   # for k in range(len(df_dummy)):
    #    if df_dummy.iloc[k]['date']==start:
     #       if k==0:
      #          date_list.append(df_dummy.iloc[k]['date'])
       #         continue
        #    else:
         #       break
        #date_list.append(df_dummy.iloc[k]['date'])
        
            
    #for k in range(len(df_dummy)):
     #   date_list.append(df_dummy.iloc[k]['date'])
        
    
    b=[]
    h=0
    for ticker in tickers:
        a=[]
        #print(ticker)
        h=h+1
        if h%10==0:
            print(h)
        for k in range (len(df_dummy)):
            #print(k)
            if df_dummy.iloc[k]['Ticker']==ticker:
                a.append(df_dummy.iloc[k]['PX_LAST'])
        b.append(a)  
    df_dummy2=pd.DataFrame(b,columns=date_list)
    
    df_dummy2["ticker"]=tickers
    df_dummy2.set_index('ticker',inplace=True)
    df_dummy2.reset_index(inplace=True)
    
    print('df2_dummy formed, now calculating p_score')
    
    #p_score calculations
    warnings.filterwarnings("ignore")
    
    c=[]
    for k in range (len(df_dummy2)):
        d=[]
        for j in range (len(df_dummy2)):
            score,pvalue,_=coint(list(df_dummy2.iloc[k][1:]),list(df_dummy2.iloc[j][1:]))
            d.append(pvalue)
        c.append(d)
    df_dummy3=pd.DataFrame(c,columns=tickers)
    
    print('pscore calculation done and stored in df_dummy3')
    
    df_dummy3["ticker"]=tickers
    df_dummy3.set_index('ticker',inplace=True)
    
    threshold = 0.01
    df_dummy4=df_dummy3[df_dummy3[:][:]<threshold]
    df_dummy4.reset_index(inplace=True)
    df_dummy4.drop("ticker",inplace=True,axis=1)
    
    print('now forming pairs based on threshold!')
    
    pairs={}
    for k in range(len(df_dummy4)):
        x=[]
        for j in range (len(tickers)):
            if df_dummy4.iloc[k][tickers[j]]>0:
                x.append(tickers[j])
        pairs[tickers[k]]=x
    
    print('pairs dictionary formed, now removing duplicates')
    
    empty_keys = [k for k,v in pairs.items() if not v]
    for k in empty_keys:
        del pairs[k]
        
    final=[]
    for key in pairs:
        #t=[]
        for k in range (len(pairs[key])):
            y=[]
            y.append(key)
            y.append(pairs[key][k])
            #t.append(y)
            final.append(y)
    
    rep=[]
    for k in range(len(final)):
        for j in range (k+1,len(final)):
            if final[k][0]==final[j][1]:
                if final[k][1]==final[j][0]:
                    rep.append(j)
                    
    for index in sorted(rep, reverse=True):
        del final[index]
        
    final_pairs[start]=final
    
    
    del df_dummy
    del df_dummy2
    del df_dummy3
    del df_dummy4
    del df_forDate
    
    print(i)

winsound.Beep(500,2000)

Start dates taken!
Other dates removed from df_dummy
10
20
30
40
df2_dummy formed, now calculating p_score
pscore calculation done and stored in df_dummy3
now forming pairs based on threshold!
pairs dictionary formed, now removing duplicates
12
Start dates taken!
Other dates removed from df_dummy
10
20
30
40
df2_dummy formed, now calculating p_score
pscore calculation done and stored in df_dummy3
now forming pairs based on threshold!
pairs dictionary formed, now removing duplicates
13
Start dates taken!
Other dates removed from df_dummy
10
20
30
40
df2_dummy formed, now calculating p_score
pscore calculation done and stored in df_dummy3
now forming pairs based on threshold!
pairs dictionary formed, now removing duplicates
14
Start dates taken!
Other dates removed from df_dummy
10
20
30
40
df2_dummy formed, now calculating p_score


KeyboardInterrupt: 

convert the dictionary keys to standard datetime format

In [34]:
for key in final_pairs:
    key = pd.to_datetime(key,dayfirst=True)
    #print(datetimeObj)

making a pickle file and storing the pairs dictionary into it

In [36]:
import pickle
pickle_out = open("dict.pickle","wb")
pickle.dump(final_pairs, pickle_out)
pickle_out.close()